## Tensorflow2 API로 모델 작성하기: MNIST (1) Sequential API 활용

### TensorFlow2 API 알아보기

### TensorFlow2에서 딥러닝 모델을 작성하는 방법 3가지

#### 1. Sequential
- 입력부터 출력까지 레이어를 그야말로 sequential하게 차곡차곡 add해서 쌓아나가기만 하면 됩니다. 무엇보다 이 방식은 초보자가 접근하기에 매우 쉽다는 장점이 있습니다. 그렇지만 모델의 입력과 출력이 여러 개인 경우에는 적합하지 않은 모델링 방식입니다. 
- Sequential 모델의 제약점은 반드시 입력 1가지, 출력 1가지를 전제로 한다는 것입니다.

#### 2. Functional
- Sequential Model을 활용하는 것과 다른 점은 바로 keras.Model을 사용한다는 점입니다. 그래서 Sequential Model을 쓰는 것보다 더 일반적인 접근인 것입니다. Sequential Model이란 사실 keras.Model을 상속받아 확장한 특수사례에 불과한 것이니까요. Functional API를 활용하면 앞서 배운 Sequential Model을 활용하는 것보다 더 자유로운 모델링을 진행할 수 있습니다. 
- Functional이라는 뜻이 뭔가요? 함수형으로 모델을 구성한다는 것, 즉 입력과 출력을 규정함으로써 모델 전체를 규정한다는 생각입니다. 그래서 이번에는 Input이라는 것을 규정합니다. Input이 될 수 있는 텐서가 여러개가 될 수 도 있습니다. 그리고 레이어들을 자유롭게 엮어 출력(Output)까지 규정하면 Model이란 바로 inputs와 outputs 만으로 규정됩니다. 정말 Functional하지 않나요? 
- Functional API를 통해 다중입력/출력을 가지는 모델을 구성할 수 있습니다.

#### 3. Model Subclassing
- Subclassing은 클래스로 구현된 기존의 모델을 상속받아 자신만의 모델을 만들어나가는 방식입니다. 마지막으로 Subclassing을 활용하면 제일 자유로운 모델링을 진행할 수 있습니다. 
- 사실 본질적으로는 Functional한 접근과 차이가 없습니다. 이것은 keras.Model을 상속받은 모델 클래스를 만드는 것이기 때문입니다. 처음 만났던 Sequential Model도 따지고 보면 keras.Model을 상속받은 모델 클래스의 하나일 뿐입니다. 
- keras.Model은 위와 같이 __init__()이라는 메소드 안에서 레이어 구성을 정의합니다. 그리고 call()이라는 메소드 안에서 레이어간 forward propagation을 구현합니다. 이것으로 끝입니다. 
- Functional 방식과 비교하자면, call()의 입력이 Input이고, call()의 리턴값이 Output이 되는 것입니다.
- 다만, 각 레이어에 대한 깊은 이해가 필요하고 초심자에게 의도치 않은 버그를 유발할 수 있습니다. 그렇지만 여러분들이 공부해 나가시면서 복잡한 모델링을 진행하시게 되면 가장 많이 접하게 되실 모델링 스타일이기에 실습 세션에서 다뤄볼 예정입니다.


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [3]:
# Sequential Model 구성

model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu'),  # 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
    keras.layers.Conv2D(64, 3, activation='relu'),  # 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
    keras.layers.Flatten(),  # Flatten 레이어
    keras.layers.Dense(128, activation='relu'),  # 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
    keras.layers.Dense(10, activation='softmax')  # 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
])

In [4]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1059 - accuracy: 0.9680
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0343 - accuracy: 0.9891
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0201 - accuracy: 0.9937
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0128 - accuracy: 0.9957
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0093 - accuracy: 0.9970
313/313 - 3s - loss: 0.0438 - accuracy: 0.9883


[0.043818481266498566, 0.9883000254631042]

## Tensorflow2 API로 모델 작성하기: MNIST (2) Functional API 활용

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [7]:
inputs = keras.Input(shape=(28, 28, 1))  # (28X28X1) 차원으로 정의된 Input

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)  # 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
x = keras.layers.Conv2D(64, 3, activation='relu')(x)  # 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
x = keras.layers.Flatten()(x)  # Flatten 레이어
x = keras.layers.Dense(128, activation='relu')(x)  # 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
predictions = keras.layers.Dense(10, activation='softmax')(x)  # 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)

model = keras.Model(inputs=inputs, outputs=predictions)

In [8]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1062 - accuracy: 0.9675
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0336 - accuracy: 0.9896
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0209 - accuracy: 0.9927
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0127 - accuracy: 0.9958
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0099 - accuracy: 0.9965
313/313 - 0s - loss: 0.0424 - accuracy: 0.9895


[0.042362187057733536, 0.9894999861717224]

## Tensorflow2 API로 모델 작성하기: MNIST (3) Subclassing 활용

In [10]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [11]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [12]:
# Subclassing을 활용한 Model 구성

class CustomModel(keras.Model):  # keras.Model 을 상속받았으며, __init__()와 call() 메소드를 가진 모델 클래스
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')  # 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')  # 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
        self.flatten = keras.layers.Flatten()  # Flatten 레이어
        self.fc1 = keras.layers.Dense(128, activation='relu')  # 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
        self.fc2 = keras.layers.Dense(10, activation='softmax')  # 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)

    def call(self, x):  # call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

In [13]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1058 - accuracy: 0.9674
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0348 - accuracy: 0.9891
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0190 - accuracy: 0.9940
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0130 - accuracy: 0.9952
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0081 - accuracy: 0.9975
313/313 - 0s - loss: 0.0509 - accuracy: 0.9873


[0.05089791119098663, 0.9872999787330627]

## TensorFlow2 API로 모델 작성 및 학습하기: CIFAR-100 (1) Sequential API 활용

In [14]:
import tensorflow as tf
from tensorflow import keras

In [17]:
# 데이터 구성부분 : 데이터 저장 디렉토리 ~/.keras/datasets/cifar-100-python
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [18]:
# Sequential Model 구성
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),  # 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
    keras.layers.MaxPool2D((2,2)),  # pool_size가 2인 MaxPool 레이어
    keras.layers.Conv2D(32, 3, activation='relu'),  # 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
    keras.layers.MaxPool2D((2,2)),  # pool_size가 2인 MaxPool 레이어
    keras.layers.Flatten(),  # Flatten 레이어
    keras.layers.Dense(256, activation='relu'),  # 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
    keras.layers.Dense(100, activation='softmax')  # 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
])

In [19]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 8s 5ms/step - loss: 3.6035 - accuracy: 0.1595
Epoch 2/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.9003 - accuracy: 0.2835
Epoch 3/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.5976 - accuracy: 0.3460
Epoch 4/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.3813 - accuracy: 0.3908
Epoch 5/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.2163 - accuracy: 0.4281
313/313 - 0s - loss: 2.5519 - accuracy: 0.3664


[2.5518550872802734, 0.36640000343322754]

## Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (2) Functional API 활용

이전 스텝과 큰 차이가 없는 내용입니다. 다만, 이번에는 keras.Model을 직접 활용하여야 하므로, keras.Input으로 정의된 input및 output 레이어 구성을 통해 model을 구현하셔야 합니다.

In [20]:
import tensorflow as tf
from tensorflow import keras

In [21]:
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [22]:
# Functional API를 활용한 Model 구성

inputs = keras.Input(shape=(32, 32, 3))  # (32X32X3) 차원으로 정의된 Input

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)  # 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
x = keras.layers.MaxPool2D((2,2))(x)  # pool_size가 2인 MaxPool 레이어
x = keras.layers.Conv2D(32, 3, activation='relu')(x)  # 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
x = keras.layers.MaxPool2D((2,2))(x)  # pool_size가 2인 MaxPool 레이어
x = keras.layers.Flatten()(x)  # Flatten 레이어
x = keras.layers.Dense(256, activation='relu')(x)  # 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
predictions = keras.layers.Dense(100, activation='softmax')(x)  # 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)

model = keras.Model(inputs=inputs, outputs=predictions)

In [23]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 2s 1ms/step - loss: 3.6305 - accuracy: 0.1531
Epoch 2/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.9265 - accuracy: 0.2798
Epoch 3/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.6234 - accuracy: 0.3407
Epoch 4/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.4158 - accuracy: 0.3844
Epoch 5/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.2596 - accuracy: 0.4168
313/313 - 0s - loss: 2.5849 - accuracy: 0.3516


[2.584912061691284, 0.3515999913215637]

## Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (3) Subclassing 활용

In [24]:
import tensorflow as tf
from tensorflow import keras

In [25]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


In [26]:
# Subclassing을 활용한 Model 구성

class CustomModel(keras.Model):  # keras.Model 을 상속받았으며, __init__()와 call() 메소드를 가진 모델 클래스
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')  # 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
        self.maxpool1 = keras.layers.MaxPool2D((2,2))  # pool_size가 2인 MaxPool 레이어
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')  # 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
        self.maxpool2 = keras.layers.MaxPool2D((2,2))  # pool_size가 2인 MaxPool 레이어
        self.flatten = keras.layers.Flatten()  # Flatten 레이어
        self.fc1 = keras.layers.Dense(256, activation='relu')  # 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
        self.fc2 = keras.layers.Dense(100, activation='softmax')  # 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)

    def call(self, x):  # call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

In [27]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 2s 1ms/step - loss: 3.6185 - accuracy: 0.1546
Epoch 2/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.9201 - accuracy: 0.2792
Epoch 3/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.6124 - accuracy: 0.3414
Epoch 4/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.4091 - accuracy: 0.3862
Epoch 5/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.2459 - accuracy: 0.4194
313/313 - 0s - loss: 2.6224 - accuracy: 0.3484


[2.622358798980713, 0.3483999967575073]

## GradientTape의 활용

### Automatic differentiation - GradientTape
우리는 조금 전까지 아주 비슷한 테스크 2개를, 본질적으로 큰 차이가 없는 3개의 모델 구성 방법을 활용하여 딥러닝으로 구현해 보았습니다. 그동안 완전히 동일하게 구성했던 것은 바로 "모델 학습" 관련 부분입니다.

### Numpy만 가지고 딥러닝을 구현하는 것을 회상해 봅시다. 

model.fit()이라는 한줄로 수행 가능한 딥러닝 모델 훈련 과정은 실제로는 어떠했나요?

- Forward Propagation 수행 및 중간 레이어값 저장
- Loss 값 계산
- 중간 레이어값 및 Loss를 활용한 체인룰(chain rule) 방식의 역전파(Backward Propagation) 수행
- 학습 파라미터 업데이트

이상 4단계로 이루어진 train_step 을 여러번 반복했습니다.

이런 과정이 TF2 API에는 model.fit()이라는 메소드 안에 모두 추상화되어 감추어져 있습니다.

Tensorflow에서 제공하는 tf.GradientTape는 위와 같이 순전파(forward pass)로 진행된 모든 연산의 중간 레이어값을 tape에 기록하고, 이를 이용해 gradient를 계산한 후 tape를 폐기하는 기능을 수행합니다. 그러면 아래에서는 이전 스텝에서 진행했던 학습을 tf.GradientTape를 이용한 것으로 변형해 보겠습니다. tf.GradientTape는 이후 그래디언트를 좀더 고급스럽게 활용하는 다양한 기법을 통해 자주 만나게 될 것입니다.

In [28]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

50000 10000


In [29]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [30]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            if step % batch_size == batch_size-1:
                x_batch.append(x)
                y_batch.append(y)
                loss = train_step(np.array(x_batch), np.array(y_batch))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0: last batch loss = 4.5174
Epoch 1: last batch loss = 4.0315
Epoch 2: last batch loss = 4.0218
Epoch 3: last batch loss = 3.5991
Epoch 4: last batch loss = 3.1201
It took 58.800514459609985 seconds


In [31]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 0s 2ms/step


0.0597